## Integration Testing


One of the most common ways for conducting integration tests is conducting API tests. This notebook talk about how to use [pytest](https://docs.pytest.org/en/stable/) along with [requests](https://requests.readthedocs.io/en/master/) to automated tests on a RESTFul API.

For this example, we will be using the [json-server](https://jsonplaceholder.typicode.com/) and run it locally for tests (as the public one ignore the POSTS and PUTS). To install do ```npm install -g json-server``` and run it in the background with ```json-server --watch db.json```. This should startup a RESTFul server in ```http://localhost:3000``` that will be run tests on. 

### Some notes on json-server
[json-server](https://jsonplaceholder.typicode.com/) uses [lowdb](https://github.com/typicode/lowdb) which is a file-based json database. If you run ```json-server --watch db.json``` and check the contents of ```db.json``` as you test, you should see the database contents.

[json-server](https://jsonplaceholder.typicode.com/) is not just a fake RESTFul API server that you play around with. It can also be used to create custom mock RESTFul API's as well and it also plays along with [Faker.js](https://github.com/marak/Faker.js/) to generate the mock data for its database. If you want to know more about how to to this visit: https://github.com/adelagon/javascript-nodejs-gitbook/blob/master/qe/mock.md

### Automating API Tests

The code example below build upon what we have learned on Unit Testing notebook and broadens our use of [pytest](https://docs.pytest.org/en/stable/) for integration testing along with [requests](https://requests.readthedocs.io/en/master/) to make API calls and [Faker](https://pypi.org/project/Faker/) for mocking data inputs.

We will build a set of tests on the json-server's [users endpoint](https://jsonplaceholder.typicode.com/users).

First you need to run the code cell below to enable the execution of pytest within jupyter.

In [ ]:
import ipytest
ipytest.autoconfig()

Sometimes when writing tests, you need to share values that were derived from previous tests to other tests. This can be done either by fixtures or using [conftest.py](https://docs.pytest.org/en/2.7.3/plugins.html?highlight=re). The conftest.py is primary used for working with pytest plugins but we can also use it to also store shared variables (See Storage class) and to externalize test configurations (See Config class).

In order to run the next few test, please make sure that you execute the code cell below:

In [ ]:
# %load src/conftest.py
class Storage:
    """
    Shared variables
    """
    user_id = None
    user_name = None
    response = None

class Config:
    """
    Test configurations
    """
    users_endpoint = "http://localhost:3000/users"
    user_endpoint = "http://localhost:3000/users/{}"


Once the ```conftest.py``` has been loaded, you may now run the pyunit test below. Please ignore the first line as it is only needed in order to run pytest within jupyter. Docstrings has been provided for the code run through:

In [ ]:
%%run_pytest[clean] -svv
import json
import pytest
import requests
import operator
from faker import Faker

@pytest.fixture(scope="module")
def values():
    """
    Generate random values. The values fixture
    can be referenced by any test function within
    module
    """
    fake = Faker()
    yield {
        "name": fake.name(),
        "username": fake.email().split('@')[0],
        "email": fake.company_email(),
        "phone": fake.phone_number(),
        "website": fake.domain_name(),
        "address": {
            "street": fake.street_address(),
            "suite": fake.building_number(),
            "city": fake.city(),
            "zipcode": fake.postcode()
        },
        "company": {
            "name": fake.company(),
            "catchPhrase": fake.catch_phrase(),
            "bs": fake.bs()
        }
    }
    ### Any line of code beyond here can be used for teardown

def test_add_user(values):
    """
    Test POST /users
    """
    r = requests.post(
        Config.users_endpoint,
        data=json.dumps(values),
        headers={'Content-Type': 'application/json'}
    )
    assert r.status_code == 201
    # Store the user_id provided the API for the next few tests
    Storage.user_id = r.json()['id']

def test_update_user(values):
    """
    Test PUT /users/{id}
    """
    # Generate new name
    values['name'] = Faker().name()
    # Update the name of the previously created user
    r = requests.put(
        Config.user_endpoint.format(Storage.user_id),
        data=json.dumps(values),
        headers={'Content-Type': 'application/json'}
    )
    assert r.status_code == 200
    assert r.json()['name'] == values['name']
    # Store the new user_name for the new test
    Storage.user_name = r.json()['name']

def test_get_user(values):
    """
    Test GET /users/{id}
    """
    # Fetch the previously created user
    r = requests.get(
        Config.user_endpoint.format(Storage.user_id),
    )
    assert r.status_code == 200
    # Check if the name has been updated
    assert Storage.user_name == r.json()['name']
    # Let's add id in values for the equality check
    values['id'] = r.json()['id']
    assert operator.eq(values, r.json()) == True
    
def test_delete_user():
    """
    Test DELETE /users/{id}
    """
    r = requests.delete(
        Config.user_endpoint.format(Storage.user_id)
    )
    assert r.status_code == 200
    # Check if the user was really deleted
    r = requests.get(
        Config.user_endpoint.format(Storage.user_id)
    )
    assert r.status_code == 404



### Behavior-Driven Development (OPTIONAL)

Some engineering teams have embraced [Behavior-Driven Development (BDD)](https://en.wikipedia.org/wiki/Behavior-driven_development) which encourages collaboration between non-technical personnel (business, product, and traditional QA's) and engineering team that is facilitated by writing together an human-readable document or in other terms a simple Domain-Specific Language (DSL). This document is then used to write software and tests. Often, these documents also serves as user stories.

One of the popular DSL's for BDD is the [Gherkin Syntax](https://cucumber.io/docs/gherkin/) where in features are spec'd in human readable syntax such as:

```
Scenario: Eric wants to withdraw money from his bank account at an ATM
    Given Eric has a valid Credit or Debit card
    And his account balance is $100
    When he inserts his card
    And withdraws $45
    Then the ATM should return $45
    And his account balance is $55
```

If you and your team plans to implement is method of working, engineers can use the features written in Gherkin Syntax to write tests. [pytest](https://docs.pytest.org/en/stable/) has a mature extension for this called [pytest-bdd](https://github.com/pytest-dev/pytest-bdd).

For the API test above, we can derive a simple Gherkin code like so:

```
Feature: users CRUD methods
    As an API user, I should be able to ADD,
    UPDATE, DELETE, and GET users.

    Scenario: Add User
        Given the Users API is sent a random user profile using "POST" method
        Then the response status code is "201"
        And the response contains an "id" field
    
    Scenario: Update User
        Given an existing user, I update its profile with a random "name" using "PUT" method
        Then the response status code is "200"
        And the response "name" field should be equal to the random name

    Scenario: Get User
        Given an existing user, I will get its profile using its "id" and "GET" method
        Then the response status code is "200"
        And the response body should be equal to the latest version of the profile after update

    Scenario: Delete User
        Given an existing user, I will delete it using its "id" and "DELETE" method
        Then the response status code is "200"
        Given the deleted user, I will attempt to get its profile using its "id" and "GET" method
        Then the response status code is "404"
```

The code for this is stored in ```src/features/users_crud.feature```

With [pytest-bdd](https://github.com/pytest-dev/pytest-bdd), you can automatically generate boilerplate code for tests by running ```pytest-bdd generate src/features/users_crud.feature```.

The full implementation of the integration test using pytest-bdd is stored in ```src/test_users_crud.py```:

```python
# coding=utf-8
"""users CRUD methods feature tests."""
import json
import pytest
import requests
import operator
from faker import Faker
from pytest_bdd import (
    given,
    scenarios,
    then,
    when,
    parsers
)

from conftest import Config, Storage

# Load scenarios from the feature file
scenarios('features/users_crud.feature')

@pytest.fixture(scope="module")
def values():
    """
    Generate random values
    """
    fake = Faker()
    yield {
        "name": fake.name(),
        "username": fake.email().split('@')[0],
        "email": fake.company_email(),
        "phone": fake.phone_number(),
        "website": fake.domain_name(),
        "address": {
            "street": fake.street_address(),
            "suite": fake.building_number(),
            "city": fake.city(),
            "zipcode": fake.postcode()
        },
        "company": {
            "name": fake.company(),
            "catchPhrase": fake.catch_phrase(),
            "bs": fake.bs()
        }
    }
    ### Cleanup

### This specific function is reused by multiple scenarios due to the parser match
@then(parsers.parse('the response status code is "{status:d}"'))
def the_response_status_code_is_(status):
    """
    This gets reused for every stratus code check
    """
    assert Storage.response.status_code == status

### Helper Function for sending requests
def send_request(method, endpoint, id=None, values=None):
    if id:
        endpoint = endpoint.format(id)
    if values:
        r = requests.request(
            method,
            endpoint,
            data=json.dumps(values),
            headers={'Content-Type': 'application/json'}
        )
    else:
        r = requests.request(
            method,
            endpoint,
        )
    Storage.response = r

### Scenario: Add User
@given(parsers.parse('the Users API is sent a random user profile using "{method}" method'))
def add_user(values, method):
    """
    It is important to note the {method} on the decorator. This allows to collect parameters from 
    the feature file. In this case, the HTTP method being used is dynamically injected by the
    feature file (in case you want to change HTTP methods)
    """
    send_request(method, Config.users_endpoint, id=Storage.user_id, values=values)

@then(parsers.parse('the response contains an "{required_field}" field'))
def the_response_contains_an_id_field(required_field):
    # Save User id for the next scenarios
    Storage.user_id = Storage.response.json()['id']
    assert required_field in Storage.response.json()

### Scenario: Update User
@given(parsers.parse('an existing user, I update its profile with a random "name" using "{method}" method'))
def update_user_response(values, method, faker):
    values['name'] = faker.name()
    send_request(method, Config.user_endpoint, id=Storage.user_id, values=values)

@then(parsers.parse('the response "{field}" field should be equal to the random name'))
def the_response_name_field_should_be_equal_to_the_random_name(values, field):
    assert Storage.response.json()[field] == values['name']

### Scenario: Get User
@given(parsers.parse('an existing user, I will get its profile using its "id" and "{method}" method'))
def get_user_response(method):
    send_request(method, Config.user_endpoint, id=Storage.user_id)

@then('the response body should be equal to the latest version of the profile after update')
def the_response_body_should_be_equal_to_the_latest_version_of_the_profile_after_update(values):
    values['id'] = Storage.response.json()['id']
    assert operator.eq(values, Storage.response.json()) == True

### Scenario: Delete User
@given(parsers.parse('an existing user, I will delete it using its "id" and "{method}" method'))
def delete_user_response(method):
    send_request(method, Config.user_endpoint, id=Storage.user_id)

@given(parsers.parse('the deleted user, I will attempt to get its profile using its "id" and "{method}" method'))
def get_deleted_user(method):
    send_request(method, Config.user_endpoint, id=Storage.user_id)

```

Unfortunately, pytest-bdd cannot be run in jupyter directly. To run it youself do:

```
cd src
pytest -vv
```


